# JETANK ColorTracking

This document is used to implement a color tracking function, which tracks bright yellow objects by default.

**Note: This document include servos control. When running the code, be careful if there are fragile objects in the range of motion of the robot arm, and keep it away from children. **

# Import camera function libraries

After running the following code block, wait a while and wait for the camera to initialize. After the initialization is successful, a 300x300-sized real-time video screen will appear below the code block.

You can right-click on this screen and click `Create New View for Output`, so that you can place the camera screen in the window again. Even if you browse to other part of the document, you can still watch the camera screen at any time. This method applies to other widgets.

The initialization may fail due to running this code block multiple times. The solution is already included in `jetbot.Camera`, you only need to restart the Kernel, but be careful not to use the circle arrow above the tab, chances are the camera will still fail to initialize.

It is a recommended method to restart the Kernel:
In `File Browser` on the left, right-click on the `*.ipynb` file with a green dot in front (the green origin indicates that the Kernel is running), select `Shut Down Kernel`, and you will find a green dot disappears, then close this tab and double-click the `*.ipynb` file that was closed just now to restart the kernel.

Run the following code again, and the camera should be initialized normally.

In [1]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

image_widget = ipywidgets.Image()  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…


### Importing the TTLServo class

First, we need to import the library used to control the servos. Before running this code, please note that ttyTHS1 (the serial port for communication between Jetson Nano and the servos) has been occupied by the Jetson Nano system itself by default. If the installation script executes everything normally, it will shut down the service that previously occupied ttyTHS1, and the script will automatically add the 0666 permissions to ttyTHS1.



If the previous installation script fails to execute, you may encounter a Permission Denied error when using ttyTHS1 here. If you encounter this error, the following is the solution:
- Click the plus sign in the upper left corner of JupyterLab, a new Launcher tab will open, and Terminal in this tab will open the Jetson Nano console.
- Enter the following in the console and press Enter.
    > sudo chmod 777 /dev/ttyTHS1
- Enter Jetpack's default password `jetbot`, and press Enter to confirm the modification permissions.
- Right-click on `JETANK_1_servos.ipynb` on the left, and click `Shut Down Kernal`.
- Close the current `JETANK_1_servos.ipynb` tab and double-click on the left side `JETANK_1_servos.ipynb` again to run a new Kernal.

Then you can select the following code and press Ctrl+Enter to import the library used to control the servos. The following prompt will appear:
Succeeded to open the port
Succeeded to change the baudrate

In [2]:
from SCSCtrl import TTLServo
import time

# If the TTL servo communicates too frequently, 
# there is a certain probability that rx and tx communication errors will be reported.
# This defines a delay for a period of time after each communication to avoid excessive communication frequency.
servoCtrlTime = 0.001

# Turn the No. 1 and No. 5 servos to the initial position.
# servo No. 1 controls the PAN axis movement of the camera pan/tilt, turning it horizontally.
# servo No. 5 controls the TILT axis movement of the camera pan/tilt, and the upward and downward pitching in the numerical direction.
TTLServo.servoAngleCtrl(1, 0, 1, 150)
time.sleep(servoCtrlTime)
TTLServo.servoAngleCtrl(5, 0, 1, 150)
time.sleep(servoCtrlTime)

# camera looks up.
def cameraUp(speedInput):
    TTLServo.servoAngleCtrl(5, -70, 1, speedInput)
    time.sleep(servoCtrlTime)

# camera looks down.
def cameraDown(speedInput):
    TTLServo.servoAngleCtrl(5, 25, 1, speedInput)
    time.sleep(servoCtrlTime)

# camera looks right.
def ptRight(speedInput):
    TTLServo.servoAngleCtrl(1, 80, 1, speedInput)
    time.sleep(servoCtrlTime)

# camera looks left.
def ptLeft(speedInput):
    TTLServo.servoAngleCtrl(1, -80, 1, speedInput)
    time.sleep(servoCtrlTime)

# camera tilt axis motion stops.
def tiltStop():
    TTLServo.servoStop(5)
    time.sleep(servoCtrlTime)

# camera pan axis motion stops.
def panStop():
    TTLServo.servoStop(1)
    time.sleep(servoCtrlTime)

# After running the above code block, the No. 1 and No. 5 servo of the camera pan/tilt will slowly rotate to the middle position.

Succeeded to open the port
Succeeded to change the baudrate



# Color recognition and tracking function

In the `ColorSelect` chapter, we introduced how to obtain the HSV value of an object color, and recorded the maximum and minimum values. You can assign the maximum and minimum values to `colorUpper `and `colorLower` , pay attention to the format: `np.array([H, S, V])`.

If you don't replace it with your own object color, you can also use the default program to realize color recognition. The default tracking color is a bright yellow object.

In [3]:
import cv2
import numpy as np

# Define the color that needs to be recognized.

#Yellow #FFFF00
colorUpper = np.array([44, 255, 255])
colorLower = np.array([24, 100, 100])

# Red FF0000
# colorUpper = np.array([180, 255, 255])
# colorLower = np.array([160, 100, 100])

# Green #00FF00
# colorUpper = np.array([50, 255, 255])
# colorLower = np.array([70, 200, 100])

# Blue #0000FF
# colorUpper = np.array([110, 225, 255])
# colorLower = np.array([135, 180, 200])

# Cyan #00FFFF
# colorUpper = np.array([80, 255, 255])
# colorLower = np.array([105, 180, 180])

# Magenta #FF00FF
# colorUpper = np.array([140, 255, 255])
# colorLower = np.array([160, 150, 200])


# Define the position tolerance of the camera when turning to this object.
# The higher the value, the higher the accuracy of the camera when aiming, 
# but too high a value may also cause the camera to continuously swing.
error_tor = 25

# This is the P value of the simple PID regulator, 
# which is the proportional adjustment coefficient of the motion speed.
# If this value is too high, it will cause the camera PT motion overshoot, 
# and if it is too low, it will cause the color tracking response speed to be too slow.
PID_P = 3

# Color recognition and tracking function.
def findColor(imageInput):
    # Convert video frames to HSV color space.
    hsv = cv2.cvtColor(imageInput, cv2.COLOR_BGR2HSV)
    
    # Create a mask for pixels that match the target color.
    mask = cv2.inRange(hsv, colorLower, colorUpper)
    
    # Erode, this process will remove the relatively 
    # small area in the mask just selected, which can be understood as denoising.
    mask = cv2.erode(mask, None, iterations=2)
    
    # dilate, the corrosion process just now will cause the large area to become 
    # smaller and the small area to disappear. This step is to restore the large area to its previous size.
    mask = cv2.dilate(mask, None, iterations=2)
    
    # Obtain the conformed area contour.
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)[-2]
    center = None
    
    # If there is a matching area, start to control the movement of the steering gear to achieve color tracking.
    if len(cnts) > 0:
        # Draw text to show that the target has been found.
        imageInput = cv2.putText(imageInput,'Target Detected',(10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
        
        # Find the contour of the largest area.
        c = max(cnts, key=cv2.contourArea)
        
        # Get the location of the center point of this area and the radius of this area.
        ((box_x, box_y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        
        # X, Y are the center points of the area.
        X = int(box_x)
        Y = int(box_y)
        
        # error_X, error_Y are the absolute value of the error 
        # between the center point of the area and the center point of the frame.
        error_Y = abs(150 - Y)
        error_X = abs(150 - X)
        
        # Draw the size and position of this area.
        cv2.rectangle(imageInput,(int(box_x-radius),int(box_y+radius)),(int(box_x+radius),int(box_y-radius)),(255,255,255),1)
        
        if Y < 150 - error_tor:
            # Camera looks up.
            imageInput = cv2.putText(imageInput,'Looking Up',(10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            cameraUp(error_Y*PID_P)
        elif Y > 150 + error_tor:
            # Camera looks down.
            imageInput = cv2.putText(imageInput,'Looking Down',(10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            cameraDown(error_Y*PID_P)
        else:
            # The error in the vertical direction is less than the tolerance, 
            # the camera stops moving in the pitch direction.
            imageInput = cv2.putText(imageInput,'Y Axis Locked',(10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            tiltStop()

        if X < 150 - error_tor:
            # Camera looks left.
            imageInput = cv2.putText(imageInput,'Looking Left',(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            ptLeft(error_X*PID_P)
        elif X > 150 + error_tor:
            # Camera looks right.
            imageInput = cv2.putText(imageInput,'Looking Right',(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            ptRight(error_X*PID_P)
        else:
            # The error in the horizontal direction is less than the tolerance, 
            # and the camera stops moving in the horizontal direction.
            imageInput = cv2.putText(imageInput,'X Axis Locked',(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            panStop()

    # If no area matching the target color is found, the camera stops rotating.
    else:
        imageInput = cv2.putText(imageInput,'Target Detecting',(10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
        tiltStop()
        panStop()
    
    return imageInput

Note that the frame you see will not change after running the following code. Only after the value of `image_widget.value` is changed and the function `camera.observe()` is called can you observe the final effect. The former value is used to display the processed frame, and the latter function is used to call the image processing related methods immediately after the new frame is collected. 

Run the following code block to turn on the color recognition and tracking function.
**Note: This document include servos control. When running the code, be careful if there are fragile objects in the range of motion of the robot arm, and keep it away from children. **

In [4]:
def execute(change):
    global image_widget
    image = change['new']
    image_widget.value = bgr8_to_jpeg(findColor(image))
    
execute({'new': camera.value})
camera.unobserve_all()
camera.observe(execute, names='value')

# Turn off this function processing and stop the camera
Run the following code block to turn off the image processing function.

In [5]:
camera.unobserve(execute, names='value')

time.sleep(1)

tiltStop()
panStop()

Again, let's close the camera conneciton properly so that we can use the camera in the later notebook.

In [6]:
camera.stop()